In [1]:
import pandas as pd
import pickle
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from time import time
import numpy as np
from keras.layers import * 
from keras.models import *
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:

data_train = pd.read_csv('trainset.data', sep=",")
data_test = pd.read_csv('testset.dat', sep=",")



In [3]:

# print(data_train['Sequence'])
# print(data_train.loc[1]['Label'])
# print(data_test.columns)
# print(data_test[' Sequence'])
data_test=data_test.rename(columns={' Sequence': 'Sequence'})
# data_test['Sequence']=data_test[]
# print(data_test['Sequence'])
# print(data_test['ID'])
# print(data_test_lables)

In [4]:
list_tripeptides=[]
list_aminoAcids=['A','R','N','D','C','E','Q','G','H','I','L','K','M','F','P','S','T','W','Y','V']
list_dipeptides=[]
for x in list_aminoAcids:
    for y in list_aminoAcids:
        list_dipeptides.append(x+y)
        for z in list_aminoAcids:
#             print(x+y+z)
            list_tripeptides.append(x+y+z)
print(len(list_tripeptides))
print(len(list_dipeptides))
list_dipeptides+=list_aminoAcids
list_tripeptides+=list_aminoAcids

8000
400


In [6]:
def get_features_peptide(df,list_peptides,k):
#     print(df.size)
    l=df.size
    df_f=pd.DataFrame(columns=list_peptides)
#     df_f.loc[0]=1
#     df_f.loc[0]['AAC']=3
#     df_f.loc[0]=df_f.loc[0]/3
    
#     print(df_f)
#     return
    for index in range(l) :
#         print(index,end=" ")
        df_f.loc[index]=0
        seq=df.loc[index]
        for i in range(len(seq)-(k-1)):
            k_pep=seq[i:i+k]
            
#             print(tri_pep)

            df_f.loc[index][k_pep]=df_f.loc[index][k_pep]+1
#     this is  for including amino acids
        for i in range(len(seq)):
            k_pep=seq[i]
            
#             print(tri_pep)

            df_f.loc[index][k_pep]=df_f.loc[index][k_pep]+1
#         print(seq)
        df_f.loc[index]=df_f.loc[index]/(len(seq)-(k-1))
#         df_f.loc[index]=df_f.loc[index][-20:]/(len(seq))
#         print(df_f.loc[index][-20:])
        
            
    return df_f
#         print(df.loc[index])
        
    
        

In [7]:

df_test_tripeptide=get_features_peptide(data_test['Sequence'],list_tripeptides,3)
df_test_tripeptide.to_pickle('df_test_tripeptide.pickle')
df_train_tripeptide=get_features_peptide(data_train['Sequence'],list_tripeptides,3)
df_train_tripeptide.to_pickle('df_train_tripeptide.pickle')

df_train_dipeptide=get_features_peptide(data_train['Sequence'],list_dipeptides,2)
df_train_dipeptide.to_pickle('df_train_dipeptide.pickle')
df_test_dipeptide=get_features_peptide(data_test['Sequence'],list_dipeptides,2)
df_test_dipeptide.to_pickle('df_test_dipeptide.pickle')

KeyboardInterrupt: 

In [8]:
print()
# df_test_tripeptide=pd.read_pickle('df_test_tripeptide.pickle')
# df_train_tripeptide=pd.read_pickle('df_train_tripeptide.pickle')
df_train_dipeptide=pd.read_pickle('df_train_dipeptide.pickle')
df_test_dipeptide=pd.read_pickle('df_test_dipeptide.pickle')
# print(df_train_tripeptide.shape,df_test_tripeptide.shape)
# print(df_train_dipeptide.shape,df_test_dipeptide.shape)
# print(df_train_dipeptide)
# for i in range(df_test_tripeptide.size):
#     if(df_test_tripeptide.min==df_test_tripeptide.loc[i]  df_test_tripeptide.loc[i]==0):
#     print(i)
#     print(df_test_tripeptide.loc[i].sum())



In [9]:
def removezerovalcolumn(df1,df):
    list_zerocol=[]
    
    for col in df.columns:
#         print(col)
        if df[col].sum()==0:
            list_zerocol.append(col)
#             print(col)

    df.drop(list_zerocol,inplace=True,axis=1)
    df1.drop(list_zerocol,inplace=True,axis=1)
    return df1,df

In [10]:
# df_train_tripeptide,df_test_tripeptide=removezerovalcolumn(df_train_tripeptide,df_test_tripeptide)
# df_train_dipeptide,df_test_dipeptide=removezerovalcolumn(df_train_dipeptide,df_test_dipeptide)
# df_test_tripeptide=removezerovalcolumn(df_test_tripeptide)
# df_train_tripeptide=removezerovalcolumn(df_train_tripeptide)
# # df_test_dipeptide=removezerovalcolumn(df_test_dipeptide)
# # df_train_dipeptide=removezerovalcolumn(df_train_dipeptide)
# print(df_train_tripeptide.shape,df_test_tripeptide.shape)
# print(df_train_dipeptide.shape,df_test_dipeptide.shape)

In [11]:
# def normalization_df(df):
#     normalized_df=(df-df.mean())/df.std()
# #     normalized_df(df-df.min())/(df.max()-df.min())

#     return normalized_df

In [12]:
# normalized_df_test_tripeptide=normalization_df(df_test_tripeptide)
# print(normalized_df_test_tripeptide)


In [13]:
# X_train=df_train_tripeptide
# X_test=df_test_tripeptide
X_train=df_train_dipeptide
X_test=df_test_dipeptide
Y_train=data_train['Label']



# print(Y_test)

# from keras.utils import to_categorical
# Y_train = to_categorical(Y_train)
# print(Y_train)
# from sklearn.preprocessing import OneHotEncoder
# onehot_encoder = OneHotEncoder(sparse=False)
# integer_encoded = Y_train.reshape(len(integer_encoded), 1)
# onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
# print(onehot_encoded)
# print(Y_train)
Y_train = pd.get_dummies(Y_train, prefix = 'Label')
# Y_train=Y_train['Label_-1']|
# print(Y_train)




In [14]:
def create_model(optimizer='adam', init='glorot_uniform'):
    model = Sequential()
    model.add(Dense(2048, input_shape=(420,)))
    model.add(Activation('relu')) 
#     model.add(Dropout(0.2)) 
#     model.add(Dropout(0.2))
    model.add(Dense(2048,init=init))
    model.add(Activation('relu'))
#     model.add(Dropout(0.2))
#     model.add(Dense(8192,init=init))
#     model.add(Activation('relu'))
#     model.add(Dropout(0.2))
#     model.add(Dense(1024,init=init))
#     model.add(Activation('relu'))
#     model.add(Dropout(0.2))
    model.add(Dense(2,init=init))
    model.add(Activation('softmax')) # This special "softmax" a
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
    return model

In [19]:

#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#train model
# model.fit(train_X, train_y, validation_split=0.2, epochs=30)



start=time()
model = KerasClassifier(build_fn=create_model)

model.fit(X_train, Y_train, validation_split=0.0, epochs=12,callbacks=[early_stopping_monitor] )
# # return
# optimizers = ['rmsprop', 'adam']
# init = ['glorot_uniform', 'normal', 'uniform']
# optimizers = ['rmsprop', 'adam']
# init = ['glorot_uniform', 'normal', 'uniform']
# epochs = np.array([8])
# batches = np.array([5, 10, 20])
# param_grid = dict(optimizer=optimizers, nb_epoch=epochs, batch_size=batches, init=init)
# grid = GridSearchCV(estimator=model, param_grid=param_grid)
# grid_result = grid.fit(X_train, Y_train)
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# for params, mean_score, scores in grid_result.grid_scores_:
#     print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))
# print("total time:",time()-start)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2048, kernel_initializer="glorot_uniform")`
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_initializer="glorot_uniform")`
  app.launch_new_instance()


Epoch 1/12
2270/2270 [==============================] - 6s 3ms/step - loss: 0.5757 - accuracy: 0.6859
Epoch 2/12
2270/2270 [==============================] - 6s 2ms/step - loss: 0.3875 - accuracy: 0.8242
Epoch 3/12
2270/2270 [==============================] - 5s 2ms/step - loss: 0.2162 - accuracy: 0.9159
Epoch 4/12
2270/2270 [==============================] - 6s 2ms/step - loss: 0.0914 - accuracy: 0.9718
Epoch 5/12
2270/2270 [==============================] - 6s 2ms/step - loss: 0.0330 - accuracy: 0.9903
Epoch 6/12
2270/2270 [==============================] - 5s 2ms/step - loss: 0.0112 - accuracy: 0.9987
Epoch 7/12
2270/2270 [==============================] - 6s 2ms/step - loss: 0.0147 - accuracy: 0.9974
Epoch 8/12
2270/2270 [==============================] - 6s 2ms/step - loss: 0.0305 - accuracy: 0.9912
Epoch 9/12
2270/2270 [==============================] - 6s 2ms/step - loss: 0.0586 - accuracy: 0.9828
Epoch 10/12
2270/2270 [==============================] - 6s 2ms/step - loss: 0.038

In [20]:
# model=grid_result
prediction=model.predict(X_test)

# print(len(prediction))
# print(prediction)



In [23]:
prediction=pd.DataFrame(data=prediction)
prediction=prediction.replace(0,-1)
new_dataframe=pd.DataFrame(columns=['ID','Label'])
new_dataframe['ID']=data_test['ID']
new_dataframe['Label']=prediction
# print(new_dataframe)
new_dataframe=new_dataframe.set_index('ID')

In [24]:
new_dataframe.to_csv('output.dat')